In [1]:
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql
import recordlinkage

## Building out the Dataframes

In [2]:
SQL = '''SELECT * from Transfers247'''
df_247 = (fx.connDBAndReturnDF(SQL)).set_index('IDYR')
df_247.index.name = '247_IDYR'
df_247

,ID,PlayerName,Year,StandardizedPosition,KeyPositionGroup
247_IDYR,,,,,
paulthurston_nebraska_2012,paulthurston_nebraska,paulthurston,2012,OL,1
gregmcmullen_nebraska_2012,gregmcmullen_nebraska,gregmcmullen,2012,DE,2
jordanwesterkamp_nebraska_2012,jordanwesterkamp_nebraska,jordanwesterkamp,2012,WR,1
imanicross_nebraska_2012,imanicross_nebraska,imanicross,2012,RB,1
tommyarmstrong_nebraska_2012,tommyarmstrong_nebraska,tommyarmstrong,2012,QB,1
...,...,...,...,...,...
dondrellharris_troy_2013,dondrellharris_troy,dondrellharris,2013,QB,1
jordanchunn_troy_2013,jordanchunn_troy,jordanchunn,2013,FB,1
bryanslater_troy_2013,bryanslater_troy,bryanslater,2013,DE,2


In [4]:
SQL = '''SELECT * from UnlinkedNCAA'''
df_ncaa = (fx.connDBAndReturnDF(SQL)).set_index('ID')
df_ncaa.index.name = 'NCAA_ID'
df_ncaa

,PlayerName,Year,StandardizedPosition,KeyPositionGroup
NCAA_ID,,,,
aadarkwah_massachusetts,aadarkwah,2017,WR,1
aadreekisconner_mississippistate,aadreekisconner,2019,DB,2
aarenvaughns_utahstate,aarenvaughns,2017,WR,1
aarionharvey_centralmichigan,aarionharvey,2020,OL,1
aaronabbott_easternmichigan,aaronabbott,2015,DB,2
...,...,...,...,...
zionjohnson_bostoncollege,zionjohnson,2019,OL,1
zionjohnson_louisville,zionjohnson,2019,OL,1
zionkelly_airforce,zionkelly,2020,DB,2


In [5]:
indexer = recordlinkage.Index()
indexer = recordlinkage.BlockIndex(on=['KeyPositionGroup'])


In [6]:
pairs = indexer.index(df_247, df_ncaa)

In [8]:
from functions import YearNFL

sumFields = []
c = recordlinkage.Compare()
c.string('PlayerName', 'PlayerName', method='damerau_levenshtein', label='PlayerName')
sumFields.append('PlayerName')
c.exact('StandardizedPosition', 'StandardizedPosition', label='StandardizedPosition')
sumFields.append('StandardizedPosition')
c.exact('KeyPositionGroup', 'KeyPositionGroup', label='KeyPositionGroup')
sumFields.append('KeyPositionGroup')
c.add(YearNFL('Year', 'Year', label='Year'))
sumFields.append('Year')




In [9]:
features = c.compute(pairs, df_247, df_ncaa)

KeyboardInterrupt: 

In [ ]:
sum = 0
for field in sumFields:
    sum = sum + features[field]

features['sum'] = sum / len(sumFields)

features = features[features['Year'] == 1.0]

features

In [ ]:
features.insert(0, 'sourceID', features.index.get_level_values(0))
features.insert(1, 'targetID', features.index.get_level_values(1))

filteredList = []
for idx, data in features.groupby(level=0):
    data = data.loc[data['Sum'].idxmax()]
    if (data['Sum'] > .5):
        filteredList.append(data)
dfFinal = pd.DataFrame()
dfFinal = dfFinal.append(filteredList)
dfFinal.to_csv("resultsNCAATransfer.csv")